<a href="https://colab.research.google.com/github/nicnicolex/KETE-HS23-WORK/blob/main/BINA_TEST_NIZI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'path/to/your/Schweizerische_Zivilluftfahrtstatistik_Passagiere_2005-2022.xlsx'
xls = pd.ExcelFile(file_path)

# Extract each sheet and save as a CSV file
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    csv_file_name = f'path/to/save/csv/{sheet}.csv'
    df.to_csv(csv_file_name, index=False)

print("CSV files created successfully.")


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

# Laden der Excel-Dateien
df1 = pd.read_excel('/content/Schweizerische_Zivilluftfahrtstatistik_Passagiere_2005-2022.xlsx', engine='openpyxl')
df2 = pd.read_excel('/content/avia_par_ch_page_spreadsheet.xlsx', engine='openpyxl')
df3 = pd.read_excel('/content/IWF.xlsx', engine='openpyxl')

# Anzeigen der ersten Zeilen jeder Datei, um die Struktur und verfügbaren Spalten zu verstehen
print("Schweizerische Zivilluftfahrtstatistik Passagiere 2005-2022:")
print(df1.head())
print("\nAvia Par CH Page Spreadsheet:")
print(df2.head())
print("\nIWF Daten:")
print(df3.head())

# Extrahieren der relevanten Daten
df1_relevant = df1[['Jahr', 'Land', 'Passagiere']]
df3_relevant = df3[['Land', 'Inflationsrate', 'BIP pro Kopf', 'Arbeitslosenquote', 'Staatsverschuldung']]

# Zusammenführen der Daten
df_merged = pd.merge(df1_relevant, df3_relevant, on='Land', how='left')

# Funktion zur Anpassung einer exponentiellen Trendlinie (Beispiel)
def exponential_growth(x, a, b, c):
    return a * np.exp(b * x) + c

# 1. Entwicklung der Passagierzahlen über die Jahre nach Land (Simpson-Paradoxon)
# Aufschlüsselung nach BIP pro Kopf
df_hoch_bip = df_merged[df_merged['BIP pro Kopf'] > df_merged['BIP pro Kopf'].median()]
df_niedrig_bip = df_merged[df_merged['BIP pro Kopf'] <= df_merged['BIP pro Kopf'].median()]

# Analyse der Trends getrennt für hohe und niedrige BIP pro Kopf
# Hier wird das Simpson-Paradoxon angewendet: Wir analysieren die Daten getrennt nach hohem und niedrigem BIP pro Kopf.
fig = px.scatter(df_hoch_bip, x='Jahr', y='Passagiere', color='Land', title='Hoher BIP pro Kopf')
fig.show()

fig = px.scatter(df_niedrig_bip, x='Jahr', y='Passagiere', color='Land', title='Niedriger BIP pro Kopf')
fig.show()

# Kombinierte Analyse
# Wenn wir die Daten kombinieren, könnten wir das Simpson-Paradoxon beobachten, wo der kombinierte Trend verschwindet oder sich umkehrt.
fig = px.scatter(df_merged, x='Jahr', y='Passagiere', color='Land', title='Kombinierte Daten')
fig.show()

# 2. Anteil der Passagiere nach Land im letzten verfügbaren Jahr (Simpson-Paradoxon)
latest_year = df_merged['Jahr'].max()
latest_data = df_merged[df_merged['Jahr'] == latest_year]

# Aufschlüsselung nach BIP-Wachstumsraten
df_hoch_wachstum = latest_data[latest_data['BIP pro Kopf'] > latest_data['BIP pro Kopf'].median()]
df_niedrig_wachstum = latest_data[latest_data['BIP pro Kopf'] <= latest_data['BIP pro Kopf'].median()]

# Analyse der Anteile getrennt für hohe und niedrige BIP-Wachstumsraten
# Hier wird das Simpson-Paradoxon angewendet: Wir analysieren die Daten getrennt nach hohen und niedrigen BIP-Wachstumsraten.
fig = px.pie(df_hoch_wachstum, values='Passagiere', names='Land', title='Hohe BIP-Wachstumsraten')
fig.show()

fig = px.pie(df_niedrig_wachstum, values='Passagiere', names='Land', title='Niedrige BIP-Wachstumsraten')
fig.show()

# Kombinierte Analyse
# Wenn wir die Daten kombinieren, könnten wir das Simpson-Paradoxon beobachten, wo der kombinierte Trend verschwindet oder sich umkehrt.
fig = px.pie(latest_data, values='Passagiere', names='Land', title='Kombinierte Daten')
fig.show()

# 3. Verteilung der Passagierzahlen nach Arbeitslosenquote (Simpson-Paradoxon)
# Aufschlüsselung nach Arbeitslosenquote
df_hoch_arbeitslos = df_merged[df_merged['Arbeitslosenquote'] > df_merged['Arbeitslosenquote'].median()]
df_niedrig_arbeitslos = df_merged[df_merged['Arbeitslosenquote'] <= df_merged['Arbeitslosenquote'].median()]

# Analyse der Verteilung getrennt für hohe und niedrige Arbeitslosenquote
# Hier wird das Simpson-Paradoxon angewendet: Wir analysieren die Daten getrennt nach hoher und niedriger Arbeitslosenquote.
fig = px.histogram(df_hoch_arbeitslos, x='Passagiere', color='Land', title='Hohe Arbeitslosenquote')
fig.show()

fig = px.histogram(df_niedrig_arbeitslos, x='Passagiere', color='Land', title='Niedrige Arbeitslosenquote')
fig.show()

# Kombinierte Analyse
# Wenn wir die Daten kombinieren, könnten wir das Simpson-Paradoxon beobachten, wo der kombinierte Trend verschwindet oder sich umkehrt.
fig = px.histogram(df_merged, x='Passagiere', color='Land', title='Kombinierte Daten')
fig.show()

# Weitere Diagramme basierend auf den Daten und Analysezielen

# Frage: Entwicklung der Passagierzahlen über die Jahre nach Land
# Diagramm: Line Chart (interaktiv) mit Slider
fig = px.line(df_merged, x='Jahr', y='Passagiere', color='Land',
              title='Entwicklung der Flugpassagierzahlen nach Land',
              labels={'Passagiere': 'Anzahl der Passagiere', 'Jahr': 'Jahr', 'Land': 'Land'},
              animation_frame="Jahr",  # Slider für Jahre
              range_y=[0, df_merged['Passagiere'].max() * 1.1])
fig.show()

# Frage: Anteil der Passagiere nach Land im letzten verfügbaren Jahr
# Diagramm: Pie Chart (interaktiv)
latest_year = df_merged['Jahr'].max()
latest_data = df_merged[df_merged['Jahr'] == latest_year]
fig = px.pie(latest_data, values='Passagiere', names='Land',
             title=f'Pie Chart: Anteil der Passagiere nach Land im Jahr {latest_year}',
             color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

# Frage: Vergleich der Passagierzahlen auf Jahresbasis
# Diagramm: Waterfall Chart (interaktiv)
fig = px.waterfall(df_merged.sort_values(by='Jahr'), x='Jahr', y='Passagiere', color='Land',
                   title='Waterfall Chart: Jährliche Veränderung der Passagierzahlen nach Land',
                   labels={'Passagiere': 'Veränderung der Passagierzahlen', 'Jahr': 'Jahr', 'Land': 'Land'})
fig.show()

# Frage: Verteilung der Passagierzahlen nach Land über die gesamte Zeitspanne
# Diagramm: TreeMap (interaktiv)
fig = px.treemap(df_merged, path=['Land'], values='Passagiere',
                 title='TreeMap: Verteilung der Passagierzahlen nach Land',
                 color='Passagiere', color_continuous_scale='Blues')
fig.show()

# Frage: Dynamische Beziehung zwischen Passagierzahlen und Jahren
# Diagramm: Bubble Chart (interaktiv)
fig = px.scatter(df_merged, x='Jahr', y='Passagiere', size='Passagiere',
                 color='Land', hover_name='Land',
                 title='Bubble Chart: Dynamik der Passagierzahlen über die Jahre',
                 animation_frame="Jahr",  # Slider für Jahre
                 size_max=60, range_x=[df_merged['Jahr'].min(), df_merged['Jahr'].max()],
                 range_y=[0, df_merged['Passagiere'].max() * 1.1])
fig.show()

# Nicht-lineare Trendlinien und Wendepunkte
for land in df_merged['Land'].unique():
    land_data = df_merged[df_merged['Land'] == land]
    xdata = land_data['Jahr'].values
    ydata = land_data['Passagiere'].values

    # Anpassung der exponentiellen Trendlinie
    popt, pcov = curve_fit(exponential_growth, xdata, ydata, maxfev=10000)
    y_fit = exponential_growth(xdata, *popt)

    # Plotten der Daten und Trendlinie
    fig = px.scatter(land_data, x='Jahr', y='Passagiere', title=f'Passagierzahlen und Trendlinie für {land}')
    fig.add_scatter(x=xdata, y=y_fit, mode='lines', name='Trendlinie')
    fig.show()

    # Identifikation von Wendepunkten
    peaks, _ = find_peaks(y_fit, prominence=1)
    troughs,


Schweizerische Zivilluftfahrtstatistik Passagiere 2005-2022:
  T5.4.1 Passagierverkehrsströme in den Landesflughafen nach Zielland der Passagiere  \
0  Courants de passagers au départ dans les aérop...                                   
1                                                NaN                                   
2                                          Jahr 2005                                   
3                                         Année 2005                                   
4                                                NaN                                   

  Unnamed: 1 Unnamed: 2                            Unnamed: 3  \
0        NaN        NaN                                   NaN   
1        NaN        NaN                                   NaN   
2        NaN      Total  Landesflughäfen\nAéroports nationaux   
3        NaN        NaN                       Basel-Mulhouse    
4        NaN        NaN                                 Total   

        Unnamed: 4 

KeyError: "None of [Index(['Jahr', 'Land', 'Passagiere'], dtype='object')] are in the [columns]"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')